# 1. Import required libs

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# 2. Init SparkSession

In [2]:
spark = SparkSession.builder.appName("HousingPricePrediction").getOrCreate()

24/11/18 10:47:10 WARN Utils: Your hostname, long-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/11/18 10:47:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/18 10:47:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 3. Load data

In [3]:
data = spark.read.csv("housing_prices.csv", header=True, inferSchema=True)

# 4. Preprocessing

## 4.1. Create features vector column

In [4]:
feature_columns = ['squareFootage', 'numRooms', 'location']  # Thay thế bằng các cột đặc trưng của bạn
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)

## 4.2. Normalize features

In [5]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withMean=True, withStd=True)
scaler_model = scaler.fit(data)
scaled_data = scaler_model.transform(data)

## 4.3. Split into TRAIN and TEST

In [6]:
train_data, test_data = scaled_data.randomSplit([0.8, 0.2])

# 5. Build  and train model

In [7]:
lr = LinearRegression(featuresCol="scaledFeatures", labelCol="price")
model = lr.fit(train_data)

24/11/18 10:47:43 WARN Instrumentation: [676b0a9c] regParam is zero, which might cause numerical instability and overfitting.


# 6. Predict and evaluate model

Error type could be RMSE, MAR or R2

In [8]:
predictions = model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Square Error (RMSE): {rmse}")

Root Mean Square Error (RMSE): 204168.1335314532
